In [2]:
# import neecessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import max,desc
#from pyspark.sql.session import SparkSession 

In [3]:
spark = SparkSession.builder.appName('architecture').getOrCreate()

In [13]:
rangedf = spark.range(1000).toDF('number')

In [14]:
divrange = rangedf.where("number % 2 = 0")

In [15]:
divrange.count()

500

In [21]:
# one liner 
print(spark.range(1000).where('id > 100').selectExpr("sum(id)").collect())

[Row(sum(id)=494450)]


In [19]:
# End to End example using Flight data


In [22]:
FlightData2015 = spark.\
                    read.option("inferSchema",'true').option("header",'true').\
                    csv('/home/amogh/Documents/spark certification/Spark-The-Definitive-Guide-master/data//flight-data/csv/2015-summary.csv')

In [23]:
FlightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [24]:
# lineage 
FlightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#48 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#48 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#46,ORIGIN_COUNTRY_NAME#47,count#48] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/amogh/Documents/spark certification/Spark-The-Definitive-Guide-maste..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [25]:
# spark creates 200 partition by default in the above lineage we can see that, but we dont need that many partitions, lets change the partition 

spark.conf.set('spark.sql.shuffle.partitions',"5")

In [28]:
FlightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#48 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#48 ASC NULLS FIRST, 5)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#46,ORIGIN_COUNTRY_NAME#47,count#48] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/amogh/Documents/spark certification/Spark-The-Definitive-Guide-maste..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [29]:
# spark sql

In [30]:
FlightData2015.createOrReplaceTempView('flight_data_2015')

sqlway = spark.sql("""
select DEST_COUNTRY_NAME, count(1) from flight_data_2015 group by DEST_COUNTRY_NAME

""")

In [32]:
sqlway.show(5)

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|             Moldova|       1|
|             Bolivia|       1|
|             Algeria|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
+--------------------+--------+
only showing top 5 rows



In [34]:
sqlway.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#46], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#46, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#46], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#46] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/amogh/Documents/spark certification/Spark-The-Definitive-Guide-maste..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [35]:
dataframeway = FlightData2015.groupBy("DEST_COUNTRY_NAME").count()

In [36]:
dataframeway.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#46], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#46, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#46], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#46] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/amogh/Documents/spark certification/Spark-The-Definitive-Guide-maste..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [37]:
# both sql and dataframe are same logical plan 

In [38]:
# we can get some intersting stats 

In [40]:
spark.sql("""

SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5

""")

DataFrame[DEST_COUNTRY_NAME: string, destination_total: bigint]

In [42]:
FlightData2015.select(max('count')).take(1)

[Row(max(count)=370002)]

In [48]:
FlightData2015.groupBy("DEST_COUNTRY_NAME").\
    sum("count").\
    withColumnRenamed("sum(count)","destination_total").\
        sort(desc("destination_total")).\
        limit(5).show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [4]:
spark.conf.get('spark.driver.host')

'172.17.0.1'

In [5]:
spark.conf.get('spark.driver.port')

'36889'